In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['font.size'] = 12
plt.rcParams['savefig.format'] = 'pdf'
sns.set_style('darkgrid')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import torch
# from PIL import Image

from scripts import *

Global seed set to 10


In [3]:
import gdown

if not Path('Video_views').exists():
    url = "https://drive.google.com/drive/folders/1F6lkq-1us0SNApoguX896iCbs-X15WVt"
    gdown.download_folder(url)

In [4]:
data = pd.read_csv('Video_views/youtube_train.csv')
print(data.shape)
data.head()

(5849, 6)


,id,Channel,Subtitles,PublishDate,Category,ViewCount
0,0,toplesofficial,('Я миллениал и я не занимаюсь сексом так част...,2017-12-31T01:35:44-08:00,Education,3019
1,1,postnauka,"(""сервитуты это одно из прямых наследие римско...",2014-09-04T08:24:47-07:00,Science & Technology,25
2,2,NaukaPRO,('[музыка] существуют сенсоры измеряющие магни...,2023-11-23T21:00:08-08:00,Science & Technology,7
3,3,postnauka,('в наше время только ленивые не говорит о том...,2014-06-23T03:49:31-07:00,Science & Technology,5
4,4,user-rb8ux1no6j,('дорогие друзья это честный рекламный ролик н...,2020-10-09T01:30:04-07:00,Education,10


In [5]:
data[data['ViewCount'] < 600].shape

(5328, 6)

In [6]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(data, train_size=0.8, random_state=3)

In [7]:
train_dataset = MyDataset(train, 'train')
val_dataset = MyDataset(val, 'val')

loading  tokenized...
loading  tokenized...


In [8]:
train_loader = DataLoader(train_dataset, num_workers=4, batch_size=50, shuffle=True)
test_loader = DataLoader(val_dataset, num_workers=4, batch_size=50, shuffle=False)

In [10]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

model_checkpoint = ModelCheckpoint(
    dirpath="saved/",
    filename="{epoch}-{val_mae:.3f}",
    monitor="val_mae",
    mode="min",
    save_top_k=1,
)


model_lightning = ModelLightning(freeze=True)

trainer = pl.Trainer(
    max_epochs=20,
    accelerator='gpu',
    gradient_clip_val=0.5,
    callbacks=[model_checkpoint],
    precision="16"
)

trainer.fit(
    model=model_lightning,
    train_dataloaders=train_loader,
    val_dataloaders=test_loader,
)

Global seed set to 10
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/media/black_chick/Notes/HSE/CV_course/cv_env/lib/python3.9/site-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | FineTuneModel     | 178 M 
1 | criterion | MSELoss           | 0     
2

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 00007: reducing learning rate of group 0 to 2.0000e-06.


Validation: 0it [00:00, ?it/s]

/media/black_chick/Notes/HSE/CV_course/cv_env/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
